In [ ]:
# 查看当前挂载的数据集目录, 该目录下的变更重启环境后会自动还原
# View dataset directory. This directory will be recovered automatically after resetting environment. 
!ls /home/aistudio/data

In [ ]:
# 查看工作区文件, 该目录下的变更将会持久保存. 请及时清理不必要的文件, 避免加载过慢.
# View personal work directory. All changes under this directory will be kept even after reset. Please clean unnecessary files in time to speed up environment loading.
!ls /home/aistudio/work

In [ ]:
# 如果需要进行持久化安装, 需要使用持久化路径, 如下方代码示例:
# If a persistence installation is required, you need to use the persistence path as the following:
!mkdir /home/aistudio/external-libraries
!pip install beautifulsoup4 -t /home/aistudio/external-libraries

In [ ]:
# 同时添加如下代码, 这样每次环境(kernel)启动的时候只要运行下方代码即可:
# Also add the following code, so that every time the environment (kernel) starts, just run the following code:
import sys
sys.path.append('/home/aistudio/external-libraries')

In [6]:
!pip install bs4

Looking in indexes: https://mirror.baidu.com/pypi/simple/
     |████████████████████████████████| 122kB 12.6MB/s eta 0:00:01
  Created wheel for bs4: filename=bs4-0.0.1-cp37-none-any.whl size=1273 sha256=889d63c892f2c23c6a7bc2162fe9b2531253a0b638155db882b591cace1f564e
  Stored in directory: /home/aistudio/.cache/pip/wheels/80/e7/37/62c60fe0c1017a55e897489ce5d5e850fa5610745d6352ed0c
Successfully built bs4


In [ ]:
EPSION = 0.0000000001

def sqrt(n):
    low,high = 1,n/2+1
    mid = (low+high) / 2
    while high-low > EPSION:
        if mid * mid > n:
            high = mid
        else:
            low = mid
        mid = (high+low) / 2
    return mid
sqrt(10)

3.162277660187101

In [15]:
# 使用request + BeautifulSoup提取12365auto投诉信息
import requests
from bs4 import BeautifulSoup
import pandas as pd

# # 请求URL
# url = 'http://www.12365auto.com/zlts/0-0-0-0-0-0_0-0-1.shtml'

# 根据request_url得到soup
def get_page_content(request_url):
    # 得到页面的内容
    header={'user-agent': 'Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/74.0.3729.131 Safari/537.36'}
    html=requests.get(request_url,headers=header,timeout=10)
    content = html.text
    #print(content)

    # 通过content创建BeautifulSoup对象
    soup = BeautifulSoup(content, 'html.parser', from_encoding='utf-8')
    return soup

# 分析当前页面的投诉
def analysis(soup):
    # 找到完整的投诉信息框
    temp = soup.find('div',class_="tslb_b")
    # 创建DataFrame
    df = pd.DataFrame(columns = ['id', 'brand', 'car_model', 'type', 'desc', 'problem', 'datetime', 'status'])
    tr_list = temp.find_all('tr')
    for tr in tr_list:
        temp = {}
        td_list = tr.find_all('td')
        if len(td_list) > 0:
            id,brand,car_model,type,desc,problem,datetime,status = td_list[0].text,td_list[1].text,td_list[2].text,td_list[3].text,td_list[4].text,td_list[5].text,td_list[6].text,td_list[7].text
            temp["id"],temp["brand"],temp["car_model"],temp["type"],temp["desc"],temp["problem"],temp["datetime"],temp["status"] = id,brand,car_model,type,desc,problem,datetime,status
            df = df.append(temp,ignore_index=True)
    return df

page_num = 20
base_url = 'http://www.12365auto.com/zlts/0-0-0-0-0-0_0-0-0-0-0-0-0-'

result = pd.DataFrame(columns = ['id', 'brand', 'car_model', 'type', 'desc', 'problem', 'datetime', 'status'])
for i in range(page_num):
    request_url = base_url+str(i+1)+'.shtml'
    soup = get_page_content(request_url)
    print(request_url)
    df = analysis(soup)
    print(df)
    result = result.append(df)
    
result.to_csv('car_complain.csv', index=False)

/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/bs4/__init__.py:220: UserWarning: You provided Unicode markup but also provided a value for from_encoding. Your from_encoding will be ignored.
  warnings.warn("You provided Unicode markup but also provided a value for from_encoding. Your from_encoding will be ignored.")


请点击[此处](https://ai.baidu.com/docs#/AIStudio_Project_Notebook/a38e5576)查看本环境基本用法.  <br>
Please click [here ](https://ai.baidu.com/docs#/AIStudio_Project_Notebook/a38e5576) for more detailed instructions. 